In [2]:
import numpy as np
import pandas as pd
import os

import kagglehub




In [3]:
# Download latest version
path = kagglehub.dataset_download("peopledatalabssf/free-7-million-company-dataset")

print(os.listdir(path))

print("Path to dataset files:", path)

#continue my code

 13%|█▎        | 35.0M/278M [00:48<05:40, 751kB/s]


KeyboardInterrupt: 

# Mount drive, import config file

In [ ]:
from google.colab import drive
import os
import sys
import importlib

drive.mount('/content/drive',force_remount = True)

project_root = f"/content/drive/MyDrive/ETH/ETH HS24/Data Science Lab/" #change to your filepath
data_path = project_root + f"data/"
config_path = os.path.join(project_root,"config/")
src_path = os.path.join(project_root,"src/")

# Add src_path
sys.path.append(src_path)

#import the config_loader script - force reload
import config_loader
importlib.reload(config_loader)
from config_loader import load_config
#load the config file
config = load_config(config_path)

Mounted at /content/drive


In [ ]:
organisation_endings = config['rules']['organisations']['organisation_endings']
print(organisation_endings)

['AG', 'GmbH', 'KG', 'OHG', 'e.K.', 'UG', 'Inc.', 'Ltd.', 'LLC', 'PLC', 'LP', 'LLP', 'SA', 'SARL', 'Sàrl', 'SAS', 'SNC', 'EURL', 'S.p.A.', 'S.r.l.', 'S.a.p.a.', 'S.n.c.', 'S.a.s.']


In [ ]:
companies_df = pd.read_csv(os.path.join(path, "companies_sorted.csv"))

In [ ]:
companies_df.head()

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960


In [ ]:
#extract all company names where country = switzerland
company_names = companies_df[companies_df["country"] == "switzerland"][['name']]
#drop na rows
company_names = company_names.dropna()
print(f"size of company_names: {len(company_names)}")
company_names['name'] = company_names['name'].str.strip().str.strip(".,;:")

size of company_names: 32742


In [ ]:
company_names.sample(10)

,name
127442,notenstein la roche privatbank ag
4349948,w. christen ag
1023990,vogel architekten
3965964,bider hauser ag
4110170,ainova ag
5602005,open interactive
1458195,montana bausysteme ag
92434,la martina
23600,eoc
774487,servette rugby club de genève


In [ ]:
import re
# Create a regex pattern that match any organisation endings
pattern = r'\b(' + '|'.join(organisation_endings) + r')\b'

# Filter the DataFrame
company_names_filtered = company_names[company_names['name'].str.contains(pattern,case = False,na = False)][['name']]
#remove any organisation_endings
company_names_filtered['name'] = company_names_filtered['name'].str.replace(pattern, '', regex=True,case = False)
company_names_filtered['name'] = company_names_filtered['name'].str.strip().str.strip('.,:;')

<ipython-input-12-78c9f304074f>:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  company_names_filtered = company_names[company_names['name'].str.contains(pattern,case = False,na = False)][['name']]


In [ ]:
print(f"size of company_names_filtered: {len(company_names_filtered)}")
company_names_filtered.sample(10)

size of company_names_filtered: 15038


,name
3575870,allocyte pharmaceuticals
2696032,speck print
2585970,silver spirits
7067389,suppliance rm
6541532,logman
6039700,moongata
1431331,fety engineering sagl
761443,ziemer ophthalmic systems
3613041,otona
1217640,corporate software


In [ ]:
#drop na
company_names_filtered = company_names_filtered.dropna()
print(f"size of company_names_filtered: {len(company_names_filtered)}")
#save dataset to drive
company_names_filtered.to_csv(os.path.join(data_path, "companies/swiss_company_names.csv"), index=False)

size of company_names_filtered: 15038
